# Federated K-Means Clustering with Scikit-learn on Iris Dataset

## Introduction to Scikit-learn, tabular data, and federated k-Means
### Scikit-learn
This example shows how to use [NVIDIA FLARE](https://nvflare.readthedocs.io/en/main/index.html) on tabular data.
It uses [Scikit-learn](https://scikit-learn.org/),
a widely used open-source machine learning library that supports supervised 
and unsupervised learning.
### Tabular data
The data used in this example is tabular in a format that can be handled by [pandas](https://pandas.pydata.org/), such that:
- rows correspond to data samples
- the first column represents the label 
- the other columns cover the features.    

Each client is expected to have one local data file containing both training 
and validation samples. To load the data for each client, the following 
parameters are expected by the local learner:
- data_file_path: string, the full path to the client's data file 
- train_start: int, start row index for the training set
- train_end: int, end row index for the training set
- valid_start: int, start row index for the validation set
- valid_end: int, end row index for the validation set

### Federated k-Means clustering
The machine learning algorithm in this example is [k-Means clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

The aggregation follows the scheme defined in [Mini-batch k-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html). 

Under this setting, each round of federated learning can be formulated as follows:
- local training: starting from global centers, each client trains a local MiniBatchKMeans model with their own data
- global aggregation: server collects the cluster center, 
  counts information from all clients, aggregates them by considering 
  each client's results as a mini-batch, and updates the global center and per-center counts.

For center initialization, at the first round, each client generates its initial centers with the k-means++ method. Then, the server collects all initial centers and performs one round of k-means to generate the initial global center.

Below we listed steps to run this example.

## Install requirements
First, install the required packages:

In [ ]:
% pip install -r code/requirements.txt

## Download and prepare data
This example uses the Iris dataset available from Scikit-learn's dataset API.  

In [ ]:
%env DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv
! python3 ./code/utils/prepare_data.py --dataset_name iris --randomize 1 --out_path ${DATASET_PATH}

This will load the data, format it properly by removing the header, order 
the label and feature columns, randomize the dataset, and save it to a CSV file with comma separation. 
The default path is `/tmp/nvflare/dataset/sklearn_iris.csv`. 

Note that the dataset contains a label for each sample, which will not be 
used for training since k-Means clustering is an unsupervised method. 
The entire dataset with labels will be used for performance evaluation 
based on [homogeneity_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html).

## Prepare clients' configs with proper data information 
For real-world FL applications, the config JSON files are expected to be 
specified by each client individually, according to their own local data path and splits for training and validation.

In this simulated study, to efficiently generate the config files for a 
study under a particular setting, we provide a script to automate the process.

Note that manual copying and content modification can achieve the same.

For an experiment with `K` clients, we split one dataset into `K+1` parts in a non-overlapping fashion: 
`K` clients' training data and `1` common validation data.

To simulate data imbalance among clients, we provided several options for client data splits by specifying how a client's data amount correlates with its ID number (from `1` to `K`):
- Uniform
- Linear
- Square
- Exponential

These options can be used to simulate no data imbalance (uniform), moderate 
data imbalance (linear), and high data imbalance (square for larger client 
number, e.g. `K=20`, exponential for smaller client number, e.g. `K=5` as 
it will be too aggressive for a larger number of clients)

This step is performed by 

In [ ]:
%env DATASET_PATH=/tmp/nvflare/dataset/sklearn_iris.csv
! python3 ./utils/prepare_job_config.py --task_name "sklearn_kmeans" --data_path "${DATASET_PATH}" --site_num 3 --valid_frac 1 --split_method "uniform"

In this example, we experiment with 3 clients under a uniform data split.

## Run simulated kmeans experiment
Now that we have the job configs ready, we run the experiment using the FL Simulator.

In [ ]:
! nvflare simulator ./code/jobs/sklearn_kmeans_3_uniform -w /tmp/nvflare/sklearn_kmeans_iris -n 3 -t 3

## Result visualization
Model accuracy is computed as the homogeneity score between the cluster formed and the ground truth label, which can be visualized in tensorboard.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/sklearn_kmeans_iris